
# Análisis de precios mensuales de alimentos, aseo e higiene por región

En este cuaderno exploramos un conjunto de datos que contiene precios promedio mensuales de
productos de alimentos, aseo e higiene por región en un país. Las columnas incluidas en el
archivo son:

- **REGION**: área geográfica en la que se registra el precio.
- **producto**: nombre del artículo (por ejemplo, aceite, azúcar, papel higiénico).
- **marca**: marca comercial del producto.
- **Promedio de Precio Nuevo**: precio promedio mensual del producto en la región (expresado en
  USD con separador decimal coma).
- **Mes**: nombre del mes en español.

Realizamos un análisis descriptivo para comprender la distribución de los precios,
visualizamos tendencias a través de los meses y proponemos un análisis de clustering para
identificar grupos de productos con patrones de precios similares.


## Carga y limpieza de datos

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ruta al archivo de datos
file_path = '../data/precios_mensuales.csv'

# Cargar datos
df = pd.read_csv(file_path)

# Limpiar la columna de precios
price_col = 'Promedio de Precio Nuevo'
# Eliminar símbolo de peso y cambiar coma por punto
df[price_col] = (
    df[price_col]
    .astype(str)
    .str.replace('$', '', regex=False)
    .str.replace(',', '.', regex=False)
    .astype(float)
)

# Mostrar las primeras filas y la info general
print(df.head())
print(df.info())


## Estadísticas descriptivas

In [ ]:

# Estadísticas descriptivas del precio
price_desc = df['Promedio de Precio Nuevo'].describe()
print(price_desc)

# Número de regiones, productos, marcas
print('Número de regiones:', df['REGION'].nunique())
print('Número de productos:', df['producto'].nunique())
print('Número de marcas:', df['marca'].nunique())
print('Meses disponibles:', df['Mes'].unique())


## Precio promedio por mes

In [ ]:

# Mapa para ordenar meses en español
month_order = ['enero','febrero','marzo','abril','mayo','junio','julio','agosto','septiembre']

mean_month = df.groupby('Mes')['Promedio de Precio Nuevo'].mean().reindex(month_order)
mean_month = mean_month.dropna()

plt.figure(figsize=(8,4))
sns.lineplot(x=mean_month.index, y=mean_month.values, marker='o')
plt.xticks(rotation=45)
plt.title('Precio promedio mensual (todos los productos y regiones)')
plt.ylabel('Precio promedio (USD)')
plt.xlabel('Mes')
plt.show()


## Precio promedio por región y mes

In [ ]:

pivot = df.pivot_table(index='Mes', columns='REGION', values='Promedio de Precio Nuevo', aggfunc='mean')
month_order = ['enero','febrero','marzo','abril','mayo','junio','julio','agosto','septiembre']
pivot = pivot.reindex(month_order)

plt.figure(figsize=(10,6))
for region in pivot.columns:
    plt.plot(pivot.index, pivot[region], marker='o', label=region)

plt.xticks(rotation=45)
plt.title('Precio promedio por región y mes')
plt.ylabel('Precio promedio (USD)')
plt.xlabel('Mes')
plt.legend(title='Región', bbox_to_anchor=(1.05,1), loc='upper left')
plt.show()


## Top 10 productos con mayor precio promedio

In [ ]:

product_means = df.groupby('producto')['Promedio de Precio Nuevo'].mean()
top_products = product_means.sort_values(ascending=False).head(10)

plt.figure(figsize=(10,6))
sns.barplot(x=top_products.values, y=top_products.index, palette='viridis')
plt.title('Top 10 productos por precio promedio')
plt.xlabel('Precio promedio (USD)')
plt.ylabel('Producto')
plt.show()


## Análisis de clustering de productos

In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Crear tabla de producto x mes
pivot = df.pivot_table(index='producto', columns='Mes', values='Promedio de Precio Nuevo', aggfunc='mean')
month_order = ['enero','febrero','marzo','abril','mayo','junio','julio','agosto','septiembre']
pivot = pivot.reindex(columns=month_order)
pivot = pivot.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)

# Estandarizar
scaler = StandardScaler()
X = scaler.fit_transform(pivot)

# KMeans con 4 clusters
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(X)

# PCA para visualización en 2D
pca = PCA(n_components=2)
components = pca.fit_transform(X)

pivot['cluster'] = clusters
pivot['PC1'] = components[:,0]
pivot['PC2'] = components[:,1]

plt.figure(figsize=(8,6))
sns.scatterplot(x='PC1', y='PC2', hue='cluster', data=pivot, palette='tab10')
plt.title('Clustering de productos por patrón de precios (KMeans, 4 clusters)')
plt.xlabel('Componente principal 1')
plt.ylabel('Componente principal 2')
plt.legend(title='Cluster')
plt.show()
